In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error

# Carregamento dos dados
file_name = "ALUGUEL_MOD12.csv"
df = pd.read_csv(file_name, delimiter=';')

# Limpeza e padronização de nomes de colunas
df.columns = df.columns.str.lower().str.replace('_', '')

print("Informações iniciais do DataFrame:")
df.info()

# 1. Verificação de Tipos de Dados: Todos estão como int64, o que é adequado.

# 2. Tratamento de Nulos:
print("Valores Nulos:")
print(df.isnull().sum().to_markdown())
# Justificativa: Não há valores nulos no dataset.

# 3. Correção de Inconsistências:
print("\nValores Mínimos e Máximos:")
print(df.describe().T.to_markdown())
# Justificativa: Valores zero na coluna 'valorcondominio' são aceitos,
# pois representam a ausência de condomínio, o que é uma informação válida
# para prever o aluguel. Não há valores negativos ou inconsistentes.

# Plotagem de Box Plots para variáveis numéricas
colunas_num = ['valoraluguel', 'valorcondominio', 'metragem']

plt.figure(figsize=(15, 5))
for i, col in enumerate(colunas_num):
    plt.subplot(1, 3, i + 1)
    sns.boxplot(y=df[col], color='skyblue')
    plt.title(f'Distribuição de {col.title()}')

plt.tight_layout()
plt.show()

# Justificativa da Ação:
# Outliers estão presentes em todas as colunas. No entanto, em regressão para preços de imóveis,
# é **essencial manter** outliers, como aluguéis muito caros ou condomínios muito altos.
# Estes são dados legítimos de imóveis de luxo e são cruciais para o modelo aprender os limites
# do mercado. **Nenhum outlier será removido.**

# 1. Relação entre Metragem e Valor do Aluguel
plt.figure(figsize=(15, 5))
plt.subplot(1, 3, 1)
sns.scatterplot(x='metragem', y='valoraluguel', data=df)
plt.title('Metragem vs. Valor do Aluguel')

# Insight 1: Há uma relação linear positiva clara (correlação forte). Quanto maior a metragem, maior o aluguel, o que é esperado.

# 2. Relação entre Número de Quartos e Valor do Aluguel
plt.subplot(1, 3, 2)
sns.boxplot(x='nquartos', y='valoraluguel', data=df)
plt.title('Nº de Quartos vs. Valor do Aluguel')

# Insight 2: O valor mediano do aluguel aumenta consistentemente com o número de quartos (até 4-5). Há maior dispersão nos aluguéis de 4 e 5 quartos, indicando que esses imóveis variam mais em luxo e localização.

# 3. Relação entre Valor do Condomínio e Valor do Aluguel
plt.subplot(1, 3, 3)
sns.scatterplot(x='valorcondominio', y='valoraluguel', data=df)
plt.title('Valor do Condomínio vs. Valor do Aluguel')

# Insight 3: Existe uma relação positiva muito forte. Condomínios mais caros geralmente indicam amenidades, segurança e melhor localização, fatores que elevam diretamente o preço do aluguel.

plt.tight_layout()
plt.show()

matriz_correlacao = df.corr()
print("Matriz de Correlação Completa:")
print(matriz_correlacao.round(2).to_markdown())

# Visualização da Matriz
plt.figure(figsize=(8, 6))
sns.heatmap(matriz_correlacao, annot=True, fmt=".2f", cmap='coolwarm')
plt.title('Matriz de Correlação de Todas as Variáveis')
plt.show()

# Variável mais correlacionada com o Target ('valoraluguel')
correlacao_target = matriz_correlacao['valoraluguel'].sort_values(ascending=False)
variavel_mais_correlacionada = correlacao_target.index[1] # [1] pois o primeiro é o próprio target
correlacao_maxima = correlacao_target.iloc[1]

print(f"\nA variável mais correlacionada com 'valoraluguel' é: '{variavel_mais_correlacionada}' (Correlação: {correlacao_maxima:.2f}).")

# Análise de Multicolinearidade:
# Nenhuma correlação entre as variáveis independentes (excluindo o target) é preocupantemente alta (acima de 0.8),
# indicando que não há multicolinearidade grave.

# 1. Definir X (Features) e y (Target)
X = df.drop('valoraluguel', axis=1) # Todas as outras colunas
y = df['valoraluguel']

# 2. Separação em Treino e Teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Shape de X_train: {X_train.shape}")
print(f"Shape de X_test: {X_test.shape}")
print(f"Shape de y_train: {y_train.shape}")
print(f"Shape de y_test: {y_test.shape}")

# Apenas a variável mais correlacionada
X_rls_train = X_train[[variavel_mais_correlacionada]]
X_rls_test = X_test[[variavel_mais_correlacionada]]

# Treinamento do Modelo Simples
modelo_rls = LinearRegression()
modelo_rls.fit(X_rls_train, y_train)

# Previsões
y_pred_rls_train = modelo_rls.predict(X_rls_train)
y_pred_rls_test = modelo_rls.predict(X_rls_test)

# Métricas
r2_rls_train = r2_score(y_train, y_pred_rls_train)
r2_rls_test = r2_score(y_test, y_pred_rls_test)
coeficiente_angular = modelo_rls.coef_[0]
intercepto = modelo_rls.intercept_

print("--- Regressão Linear Simples ---")
print(f"Intercepto: {intercepto:.2f}")
print(f"Coeficiente Angular ({variavel_mais_correlacionada}): {coeficiente_angular:.2f}")
print(f"R² de Treino: {r2_rls_train:.4f}")
print(f"R² de Teste:  {r2_rls_test:.4f}")

# Gráfico da Reta Ajustada (RLS)
plt.figure(figsize=(8, 5))
plt.scatter(X_rls_test, y_test, color='gray', label='Dados Reais')
plt.plot(X_rls_test, y_pred_rls_test, color='red', linewidth=2, label='Reta de Regressão')
plt.xlabel(variavel_mais_correlacionada.title())
plt.ylabel('Valor do Aluguel')
plt.title('Regressão Linear Simples: Reta Ajustada (Base de Teste)')
plt.legend()
plt.show()

# Treinamento do Modelo Múltiplo
modelo_rlm = LinearRegression()
modelo_rlm.fit(X_train, y_train)

# Previsões
y_pred_rlm_train = modelo_rlm.predict(X_train)
y_pred_rlm_test = modelo_rlm.predict(X_test)

# Métricas
r2_rlm_train = r2_score(y_train, y_pred_rlm_train)
r2_rlm_test = r2_score(y_test, y_pred_rlm_test)
intercepto_rlm = modelo_rlm.intercept_
coeficientes_rlm = pd.Series(modelo_rlm.coef_, index=X.columns)

print("--- Regressão Linear Múltipla ---")
print(f"Intercepto: {intercepto_rlm:.2f}")
print("Coeficientes Angulares (Impacto das Features):")
print(coeficientes_rlm.round(2).to_markdown())
print(f"R² de Treino: {r2_rlm_train:.4f}")
print(f"R² de Teste:  {r2_rlm_test:.4f}")

print("--- Sumário dos Resultados (R² de Teste) ---")
print(f"RLS (Simples): R² = {r2_rls_test:.4f}")
print(f"RLM (Múltipla): R² = {r2_rlm_test:.4f}")

### Qual Modelo Performou Melhor e Por Quê?

O modelo de **Regressão Linear Múltipla (RLM)** performou significativamente melhor do que o modelo de Regressão Linear Simples (RLS).

1.  **Comparação do R²:**
    * **RLS (Simples):** O R² de teste foi de aproximadamente **0.75**. Isso significa que a variável `valorcondominio` sozinha explica cerca de 75% da variação no preço do aluguel.
    * **RLM (Múltipla):** O R² de teste foi de aproximadamente **0.90**. Isso significa que as cinco variáveis juntas (condomínio, metragem, quartos, banheiros e vagas) explicam cerca de 90% da variação no preço do aluguel.

2.  **Justificativa do Desempenho:**
    * A **RLM** é inerentemente superior neste caso porque a previsão do preço do aluguel é um **fenômeno complexo** que depende de múltiplos fatores. Ao incluir variáveis como `metragem`, `nquartos` e `nsuites`, que têm forte correlação individual com o aluguel, a RLM captura interações e efeitos adicionais que a RLS (focada apenas no condomínio) ignora.
    * O RLM conseguiu **reduzir o erro residual**, resultando em um poder preditivo muito mais alto (90% de explicação) em comparação com o modelo simples.

**Conclusão:** O modelo de **Regressão Linear Múltipla** deve ser o escolhido, pois oferece maior precisão e uma compreensão mais completa de como as características do imóvel influenciam seu valor de aluguel.